# RL challenge

Your challenge is to learn to play [Flappy Bird](https://en.wikipedia.org/wiki/Flappy_Bird)!

Flappybird is a side-scrolling game where the agent must successfully nagivate through gaps between pipes. Only two actions in this game: at each time step, either you click and the bird flaps, or you don't click and gravity plays its role.


## Step 1 : FlappyAgentManual & FlappyAgentEasy


To begin with this Flappy Challenge, I decided to develop two very easy functions in order to understand the kind of trajectories that may make the Flappy win his challenge. We have:

* FlappyAgentManual : it is only a function that allow you to play Flappy Bird. You should press a key of your keyboard to make the bird flap. This function allow you to play Flappy and to decide the trajectory of the bird.

* FlappyAgentEasy : it is a completely deterministic function. It is not interesting for Reinforcment Learning but it is a first function that uses state variables to help Flappy to fly.


In [2]:
from ple.games.flappybird import FlappyBird
from ple import PLE
import pygame
import numpy as np
import pickle

In [3]:
# first policies

def FlappyAgentManual(state, screen):
    action = None
    events = pygame.event.get()
    for event in events:
        print(event)
        if event.type == pygame.KEYUP:
            action = 119
    return action

def FlappyAgentEasy(state, screen):
    if state['player_y'] > state['next_pipe_bottom_y'] - 50:
        action = 119
    else:
        action = None
    return action

In [ ]:
# run.py
game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=True)

p.init()
reward = 0.0

nb_games = 1
nb_data = 200
cumulated = np.zeros((nb_games))


next_pipe_bottom_y = np.array(range(nb_data))
player_vel = np.array(range(nb_data))
player_y = np.array(range(nb_data))
next_pipe_dist_to_player = np.array(range(nb_data))
count = 0
for i in range(nb_games):
    p.reset_game()
    
    while(not p.game_over()):
        state = game.getGameState()
        screen = p.getScreenRGB()
        action=FlappyAgentEasy(state, screen) 
        reward = p.act(action)
        cumulated[i] = cumulated[i] + reward
        if count <= nb_data-1:
            next_pipe_bottom_y[count] = state['next_pipe_bottom_y']
            player_vel[count] = state['player_vel']
            player_y[count] = state['player_y']
            next_pipe_dist_to_player[count] = state['next_pipe_dist_to_player']
            print(action, reward)
        count = count +1

average_score = np.mean(cumulated)
max_score = np.max(cumulated)

print(average_score)

In [ ]:
print(player_y)
print(player_vel)

As you can see, the previous code allow to begin a game and to read the values of the state of the game at every moment of the game. Through this pre-study, we can read the value that may take every variable such as **player_y**, **player_vel** or the **reward**


## FlappyAgentSarsa

Since we know the values, the next step of the challenge was to simplify the number of possible states in order to reduce the size of the space we want to explore afterwards. We define then a vector for each variable that goes from the min to the max values with a specific resolution. At the beginning we worked with around 20 possible values for each variables. 

In [4]:
# grid definition
resolution = 20
next_pipe_bottom_y = np.array(range(0,400,resolution))
player_vel = np.array(range(-10,10, 2))
player_y = np.array(range(0,400,resolution))
next_pipe_dist_to_player = np.array(range(0,300,resolution))

Then we decided to write the code of a Sarsa($\lambda$ ) algorithm. The writing of the code is quite easy, all the difficulty for the next step lies in choosing good parameters. A first idea was to take random parameters and to compare the learning converge ratio of the sarsa algorithme on the first 100 games to detect the best parameters. However my computer is quite slow so this method was too long. Here is the values I choosed and why.

* **Resolution of the grid :**
* **$\alpha$ = 0.1 :** I tried the value of the RL notebook at the begining (0.01). This parameter correponds to the compromise between the speed of convergence and the accuracy of convergence. That's why I decided to increase it until we could see improvement on the first 100 games.
* **$\gamma$ = 0.9:** It corresponds to factor of the discounted reward. It guarantees the convergemce of the reward model. So we want it tombe quite close to 1.
* **$\lambda$ = 0.9:** It corresponds
* **$\epsilon$ = 0.05:**



Other things to make our algorithm converge faster:

* **Heuristic initialisation on Q:** We used the easy policy (using difference between player_y and next_pipe_bottom_y)
* **Reward shaping:** Make more important to lose. And different type of losing.
* **Context dependent exploration:** Reducing the value of epsilon with time

In [8]:
# convert boolean to Action
def ToAction(a):
    if a:
        action = 119
    else:
        action = None
    return action

# reward shaping
def RewardShape(reward):
    if reward == 1:
        reward_ = 100
    if reward == -5:
        state_end = game.getGameState()
        reward_ = -1000 * abs(state_new['player_y'] - (state_new['next_pipe_top_y']+state_new['next_pipe_bottom_y'])/2)
    return reward_

def epsilon_greedy(Q, i,j,k, l, epsilon, state):
    a = np.argmax(Q[i][j][k][l][:])
    if(np.random.rand()<=epsilon): # random action
#if state['player_y'] > state['next_pipe_bottom_y'] - 50:
#            a = 1
#        else:
#            a = 0
        if np.random.rand()<= 0.05:
            a = 1-a
    return a

In [6]:
# parameter definition
gamma = 0.9
alpha = 0.1
epsilon = 0.1
lbd = 0.9

# initialize matrix Q and eligibility
Q = np.zeros((len(player_y), len(next_pipe_bottom_y), len(player_vel), len(next_pipe_dist_to_player) ,2))
eligibility = np.zeros((len(player_y), len(next_pipe_bottom_y), len(player_vel), len(next_pipe_dist_to_player)))

n = 100000 ##nb de parties utilisées

In [10]:
## TRAINING GAME
cumulated = np.zeros((100))
count = 0

game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=True, display_screen=True)

for kk in range(1,n):
    if ((kk+1)%1000==0):
        epsilon = epsilon/1.1
    if ((kk+1)%100 == 0):
        print('Moyenne sur les 100 derniers jeux:')
        print((5 + np.mean(cumulated)))
        cumulated = np.zeros((100))
        count = 0
    p.init()
    reward = 0.0
    p.reset_game()
    state = game.getGameState()
    screen = p.getScreenRGB()
    i = np.argmin(abs(player_y - state['player_y']))
    j = np.argmin(abs(next_pipe_bottom_y - state['next_pipe_bottom_y']))
    k = np.argmin(abs(player_vel - state['player_vel']))
    l = np.argmin(abs(next_pipe_dist_to_player - state['next_pipe_dist_to_player']))
    a = epsilon_greedy(Q, i, j, k, l, epsilon, state)
    
        
    while(not p.game_over()):
        # observe r, s and s' 
        reward = p.act(ToAction(a))
        reward_ = RewardShape(reward)
        state_new = game.getGameState()
        screen_new = p.getScreenRGB()
        i_new = np.argmin(abs(player_y - state_new['player_y']))
        j_new = np.argmin(abs(next_pipe_bottom_y - state_new['next_pipe_bottom_y']))
        k_new = np.argmin(abs(player_vel - state_new['player_vel']))
        l_new = np.argmin(abs(next_pipe_dist_to_player - state_new['next_pipe_dist_to_player']))
        aa = epsilon_greedy(Q, i_new, j_new, k_new, l_new, epsilon, state_new)
        
        eligibility = gamma * lbd * eligibility
        eligibility[i][j][k][l] = 1
        delta = reward_ + gamma*Q[i_new][j_new][k_new][l_new][aa] - Q[i][j][k][l][a] 
        Q[i][j][k][l][a] = Q[i][j][k][l][a] + alpha * eligibility[i][j][k][l]* delta
        

        i = i_new
        j = j_new
        k = k_new 
        l = l_new
        a = aa
        
        cumulated[count] = cumulated[count] + reward
    count += 1
Q


UnboundLocalError: local variable 'reward_' referenced before assignment

In [ ]:
## SAVE
with open('Qsarsa', 'wb') as f:
    pickle.dump(Q,f)

Once Q is computed, we saved it in a python file. So next time, we'll just have to import the matrix we defined the policy below:

In [ ]:
def QToPolicy(Q):
    pi = np.zeros((len(player_y), len(next_pipe_bottom_y), len(player_vel), len(next_pipe_dist_to_player)))
    for i in range(len(player_y)):
        for j in range(len(next_pipe_bottom_y)):
            for k in range(len(player_vel)):
                for l in range(len(next_pipe_dist_to_player)):
                    pi[i][j][k][l] = np.argmax(Q[i][j][k][l][:])
    return pi

def FlappyAgentSarsa(state, screen):
    
    # determine state
    i = np.argmin(abs(player_y - state['player_y']))
    j = np.argmin(abs(next_pipe_bottom_y - state['next_pipe_bottom_y']))
    k = np.argmin(abs(player_vel - state['player_vel']))
    l = np.argmin(abs(next_pipe_dist_to_player - state['next_pipe_dist_to_player']))
    
    action = ToAction(Pisarsa[i][j][k][l])
    return action

file = open("Qsarsa",'rb')
Qsarsa = pickle.load(file)
Pisarsa = QToPolicy(Qsarsa)

In [ ]:
# run.py

game = FlappyBird()
p = PLE(game, fps=30, frame_skip=1, num_steps=1, force_fps=False, display_screen=True)

p.init()
reward = 0.0

nb_games = 100
cumulated = np.zeros((nb_games))

for i in range(nb_games):
    p.reset_game()
        
    while(not p.game_over()):
        state = game.getGameState()
        screen = p.getScreenRGB()
        action=FlappyAgentSarsa(state, screen)
        
        reward = p.act(action)
        cumulated[i] = cumulated[i] + reward

average_score = np.mean(cumulated)
max_score = np.max(cumulated)

In [ ]:
print(average_score)
print(max_score)

Finally with the parameters that we show before, we managed to learn better than 15. We could improve by changing parameters, but it is quite long and we'll have more challenge in developping a more complex algorithm.

## FlappyAgentNN